## Linear Regression

## Ols

In [59]:
import numpy as np
x = np.array([1,3,5,8,10])
y = np.array([10,20,30,35,45])

### Eq -1

In [60]:
#wo = 0
#w1 = 0
#y = wo + w1*x

In [61]:
x_y_bar = (x*y).mean()
x_bar = x.mean()
y_bar = y.mean()
x_sqr_bar = (x*x).mean()

In [62]:
w1 = ((x_y_bar) - x_bar*y_bar) / (x_sqr_bar - x_bar*x_bar)
w1

3.646616541353384

In [63]:
wo = y_bar - (w1*x_bar)
wo

8.308270676691723

In [64]:
y_hat = wo+(w1*x)

In [65]:
y_hat

array([11.95488722, 19.2481203 , 26.54135338, 37.48120301, 44.77443609])

## Eq - 2

In [66]:
w1 = ((x - x_bar) * (y-y_bar)).sum() / ((x - x_bar) * (x - x_bar)).sum()
w1

3.6466165413533833

In [67]:
wo = y_bar - w1*x_bar
wo

8.30827067669173

### 1. Normal Equation Method

θ=(XTX)^−1 XT y

In [1]:
def predict_sales(x, weight, bias):
    return weight*x + bias


In [68]:
import numpy as np
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)


In [69]:
X_b = np.c_[np.ones((100, 1)), X]
X_b.shape

(100, 2)

In [4]:

theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_best

array([[4.17226905],
       [2.78631442]])

In [5]:

theta_best = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
theta_best


array([[5.84479348]])

### 2. Gradient descent

### Cost function

𝑀𝑆𝐸 =1 / 𝑁 (∑︁ 𝑛 𝑖=1 (𝑦𝑖 − (𝑚𝑥𝑖 + 𝑏))^2)

    𝑁 is the total number of observations (data points)

* 1 /  𝑁 ∑︀𝑛 𝑖=1 is the mean

*  𝑦𝑖 is the actual value of an observation and 𝑚𝑥𝑖 + 𝑏 is our prediction

In [6]:
def cost_function(x, y, weight, bias):
    companies = len(x)
    total_error = 0.0
    for i in range(companies):
        total_error += (y[i] - (weight*x[i] + bias))**2
    return total_error / companies

In [7]:
def update_weights(radio, sales, weight, bias, learning_rate):
    weight_deriv = 0
    bias_deriv = 0
    companies = len(radio)
    for i in range(companies):
        # Calculate partial derivatives
        # -2x(y - (mx + b))
        weight_deriv += -2*radio[i] * (sales[i] - (weight*radio[i] + bias))
        # -2(y - (mx + b))
        bias_deriv += -2*(sales[i] - (weight*radio[i] + bias))
        # We subtract because the derivatives point in direction of steepest ascent
        weight -= (weight_deriv / companies) * learning_rate
        bias -= (bias_deriv / companies) * learning_rate
    return weight, bias

## Training 

In [8]:
def train(radio, sales, weight, bias, learning_rate, iters):
    cost_history = []
    for i in range(iters):
        weight,bias = update_weights(radio, sales, weight, bias, learning_rate)
        #Calculate cost for auditing purposes
        cost = cost_function(radio, sales, weight, bias)
        cost_history.append(cost)
        # Log Progress
        if i % 10 == 0:
            print("iter={:d} weight={:.2f} bias={:.4f} cost={:.2}".format(i,weight, bias, cost))
    return weight, bias, cost_history


In [6]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

In [ ]:
def cost_function(features, labels, weights):
    '''
    Using Mean Absolute Error
    Features:(100,3)
    Labels: (100,1)
    Weights:(3,1)
    Returns 1D matrix of predictions
    Cost = (labels*log(predictions) + (1-labels)*log(1-predictions) ) / len(labels)
    '''
    observations = len(labels)
    predictions = predict(features, weights)
    #Take the error when label=1
    class1_cost = -labels*np.log(predictions)
    #Take the error when label=0
    class2_cost = (1-labels)*np.log(1-predictions)
    #Take the sum of both costs
    cost = class1_cost - class2_cost
    #Take the average cost
    cost = cost.sum() / observations
    return cost

In [3]:
import requests
import string
import random

# Load a list of possible words from a dictionary API or a file
response = requests.get("https://api.dictionary.com/words?limit=100")
words = response.json()

# Initialize the game with a random word and an empty set of guessed letters
target_word = random.choice(words)["word"].lower()
guessed_letters = set()

# Initialize the set of candidate words that match the length of the target word
candidate_words = set([word["word"].lower() for word in words if len(word["word"]) == len(target_word)])

# Loop until the game is won or lost
while True:
    # Print the current status of the game
    print("Target word:", " ".join([letter if letter in guessed_letters else "_" for letter in target_word]))

    # Calculate the frequency of each letter in the remaining candidate words
    frequencies = {letter: 0 for letter in string.ascii_lowercase}
    for word in candidate_words:
        for letter in set(word):
            frequencies[letter] += 1

    # Guess the most frequent letter that has not been guessed before
    guess = max(string.ascii_lowercase, key=lambda letter: (frequencies[letter], letter) not in guessed_letters)

    # Print the guessed letter
    print("Guessed letter:", guess)

    # Update the set of guessed letters
    guessed_letters.add(guess)

    # Remove all words that do not contain the guessed letter in the correct positions
    candidate_words = set([word for word in candidate_words if all(letter == guess or (letter not in guessed_letters and letter == target_word[i]) for i, letter in enumerate(word))])

    # Check if the game is won or lost
    if target_word in candidate_words:
        print("You won!")
        break
    elif len(candidate_words) == 0:
        print("You lost!")
        break

ConnectionError: HTTPSConnectionPool(host='api.dictionary.com', port=443): Max retries exceeded with url: /words?limit=100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x10c6820d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [16]:
word = "motive"
guess_words = set()

In [22]:
guess_words.add('o')

In [23]:
Hint_word = " ".join("_" if letter not in guess_words else letter for letter in word)

In [24]:
Hint_word

'm o _ _ _ _'

In [37]:
word = "motive"
def hangman_game(word):
    guess_words = set()
    Hint_word = ""
    i = 0
    while i<10:
        guess_word = input()
        if guess_word in  word:
            guess_words.add(guess_word)
            Hint_word = "".join("_" if letter not in guess_words else letter for letter in word)
        i+=1
        print("The guess word is", Hint_word)
        if str(Hint_word) == str(word):
            return "won"
            break
    return "Loss"

In [39]:
pd.DataFrame()

NameError: name 'pd' is not defined

In [242]:
## Build a machine learning model that predicts the stoppage locations the truck will stop and the time spend in each of the stops for a trip given above features
import pandas as pd
import numpy as np
data= {'trip_id':[12,13,14,14,16,54,56,75,75,82,94],
'stoppage_location':[3,2,2,1,2,4,4,1,2,4,3],
'stoppage_duration':[74,27,107,15,67,20,101,36,57,40,53],
'pickup_x':[7.7,22.7,7.4,7.4,23.8,68.3,65.2,6.9,6.9,69.3,55.4],
'pickup_y':[50.1,22.1,46.4,46.4,22.0,18.1,18.1,67.3,67.3,18.2,18.7],
'dropoff_x':[68.5,8.5,31.6,31.6,8.6,16.9,7.3,67.3,67.3,17.7,13.4],
'dropoff_y':[17.8,27.4,14.8,14.8,28.9,23.2,42.3,18.3,18.3,23.3,23.5],
'delivered_on_time':[0,0,1,1,1,1,1,0,0,1,1],
'driver':[1,1,3,3,1,2,2,2,2,2,3],
'pickup_time':[549,876,678,678,776,494,451,103,103,501,700],
'pickup_day':[37,45,54,54,65,66,49,172,172,275,354]
}
df = pd.DataFrame(data)

In [243]:
df

,trip_id,stoppage_location,stoppage_duration,pickup_x,pickup_y,dropoff_x,dropoff_y,delivered_on_time,driver,pickup_time,pickup_day
0,12,3,74,7.7,50.1,68.5,17.8,0,1,549,37
1,13,2,27,22.7,22.1,8.5,27.4,0,1,876,45
2,14,2,107,7.4,46.4,31.6,14.8,1,3,678,54
3,14,1,15,7.4,46.4,31.6,14.8,1,3,678,54
4,16,2,67,23.8,22.0,8.6,28.9,1,1,776,65
5,54,4,20,68.3,18.1,16.9,23.2,1,2,494,66
6,56,4,101,65.2,18.1,7.3,42.3,1,2,451,49
7,75,1,36,6.9,67.3,67.3,18.3,0,2,103,172
8,75,2,57,6.9,67.3,67.3,18.3,0,2,103,172
9,82,4,40,69.3,18.2,17.7,23.3,1,2,501,275


In [244]:
grouped = df.groupby("trip_id")["stoppage_location"].unique()
new_df = df.join(grouped, on="trip_id", rsuffix="_multi")
new_df

,trip_id,stoppage_location,stoppage_duration,pickup_x,pickup_y,dropoff_x,dropoff_y,delivered_on_time,driver,pickup_time,pickup_day,stoppage_location_multi
0,12,3,74,7.7,50.1,68.5,17.8,0,1,549,37,[3]
1,13,2,27,22.7,22.1,8.5,27.4,0,1,876,45,[2]
2,14,2,107,7.4,46.4,31.6,14.8,1,3,678,54,"[2, 1]"
3,14,1,15,7.4,46.4,31.6,14.8,1,3,678,54,"[2, 1]"
4,16,2,67,23.8,22.0,8.6,28.9,1,1,776,65,[2]
5,54,4,20,68.3,18.1,16.9,23.2,1,2,494,66,[4]
6,56,4,101,65.2,18.1,7.3,42.3,1,2,451,49,[4]
7,75,1,36,6.9,67.3,67.3,18.3,0,2,103,172,"[1, 2]"
8,75,2,57,6.9,67.3,67.3,18.3,0,2,103,172,"[1, 2]"
9,82,4,40,69.3,18.2,17.7,23.3,1,2,501,275,[4]


In [245]:
from sklearn.preprocessing import MultiLabelBinarizer

# create a MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

# fit the MultiLabelBinarizer on the stoppage_info list
mlb.fit(new_df['stoppage_location_multi'])
# transform the stoppage_info list to a multilabel matrix
multilabel = mlb.transform(new_df['stoppage_location_multi'])

# create a new dataframe with the multilabel columns
df_multilabel = pd.DataFrame(multilabel, columns=['stoppage_location_' + str(i) for i in range(len(mlb.classes_))])

# concatenate the original dataframe with the multilabel dataframe
df = pd.concat([df, df_multilabel], axis=1)
df

,trip_id,stoppage_location,stoppage_duration,pickup_x,pickup_y,dropoff_x,dropoff_y,delivered_on_time,driver,pickup_time,pickup_day,stoppage_location_0,stoppage_location_1,stoppage_location_2,stoppage_location_3
0,12,3,74,7.7,50.1,68.5,17.8,0,1,549,37,0,0,1,0
1,13,2,27,22.7,22.1,8.5,27.4,0,1,876,45,0,1,0,0
2,14,2,107,7.4,46.4,31.6,14.8,1,3,678,54,1,1,0,0
3,14,1,15,7.4,46.4,31.6,14.8,1,3,678,54,1,1,0,0
4,16,2,67,23.8,22.0,8.6,28.9,1,1,776,65,0,1,0,0
5,54,4,20,68.3,18.1,16.9,23.2,1,2,494,66,0,0,0,1
6,56,4,101,65.2,18.1,7.3,42.3,1,2,451,49,0,0,0,1
7,75,1,36,6.9,67.3,67.3,18.3,0,2,103,172,1,1,0,0
8,75,2,57,6.9,67.3,67.3,18.3,0,2,103,172,1,1,0,0
9,82,4,40,69.3,18.2,17.7,23.3,1,2,501,275,0,0,0,1


In [223]:
df['euc_distance'] = df.apply(lambda x: np.sqrt((x.dropoff_x - x.pickup_x)**2 + (x.dropoff_y - x.pickup_y)**2), axis=1)
df['manhattan_distance'] = df.apply(lambda x: abs(x.dropoff_x - x.pickup_x) + abs(x.dropoff_y - x.pickup_y), axis=1)

In [ ]:
df['week'] =
df['time']

In [185]:
## Problem 1
#stoppage locations prediction

In [186]:
#df['stoppage_location_']= df['stoppage_location'].apply(lambda x: str(x))
#df_ = df[["trip_id",'stoppage_location_']].groupby(["trip_id"], as_index=False).agg(lambda row: ",".join([val for val in row]))


In [187]:
#df['stoppage_location_']= df['stoppage_location'].apply(lambda x: str(x))
#df[["trip_id",'stoppage_location_']].groupby(["trip_id"], as_index=False).agg(lambda row: ",".join([val for val in row]))

In [210]:
df_cls = pd.pivot_table( data=df, 
                        index=['trip_id'], 
                        columns=['stoppage_location'], 
                        values='stoppage_duration',
                        aggfunc=['count'],
                        fill_value=0).reset_index()
df_cls.columns = ['trip_id','stoppage_location_1','stoppage_location_2','stoppage_location_3','stoppage_location_4']
#Y_classification = np.vstack((Y_classification.values))

In [211]:
df_cls

,trip_id,stoppage_location_1,stoppage_location_2,stoppage_location_3,stoppage_location_4
0,12,0,0,1,0
1,13,0,1,0,0
2,14,1,1,0,0
3,16,0,1,0,0
4,54,0,0,0,1
5,56,0,0,0,1
6,75,1,1,0,0
7,82,0,0,0,1
8,94,0,0,1,0


In [212]:
df_reg = pd.DataFrame(df.pivot(index= 'trip_id', 
        columns='stoppage_location', 
        values= 'stoppage_duration', )).fillna(0).reset_index()
df_reg.columns = ['trip_id','stoppage_location_1','stoppage_location_2','stoppage_location_3','stoppage_location_4']


In [213]:
df_reg

,trip_id,stoppage_location_1,stoppage_location_2,stoppage_location_3,stoppage_location_4
0,12,0.0,0.0,74.0,0.0
1,13,0.0,27.0,0.0,0.0
2,14,15.0,107.0,0.0,0.0
3,16,0.0,67.0,0.0,0.0
4,54,0.0,0.0,0.0,20.0
5,56,0.0,0.0,0.0,101.0
6,75,36.0,57.0,0.0,0.0
7,82,0.0,0.0,0.0,40.0
8,94,0.0,0.0,53.0,0.0


In [217]:
df_class = pd.merge(df[['trip_id','pickup_x','pickup_y','dropoff_x','dropoff_y',
                        'delivered_on_time','driver','pickup_time','pickup_day']], 
                    df_cls,
         on = 'trip_id').drop_duplicates('trip_id')

In [218]:
df_class

,trip_id,pickup_x,pickup_y,dropoff_x,dropoff_y,delivered_on_time,driver,pickup_time,pickup_day,stoppage_location_1,stoppage_location_2,stoppage_location_3,stoppage_location_4
0,12,7.7,50.1,68.5,17.8,0,1,549,37,0,0,1,0
1,13,22.7,22.1,8.5,27.4,0,1,876,45,0,1,0,0
2,14,7.4,46.4,31.6,14.8,1,3,678,54,1,1,0,0
4,16,23.8,22.0,8.6,28.9,1,1,776,65,0,1,0,0
5,54,68.3,18.1,16.9,23.2,1,2,494,66,0,0,0,1
6,56,65.2,18.1,7.3,42.3,1,2,451,49,0,0,0,1
7,75,6.9,67.3,67.3,18.3,0,2,103,172,1,1,0,0
9,82,69.3,18.2,17.7,23.3,1,2,501,275,0,0,0,1
10,94,55.4,18.7,13.4,23.5,1,3,700,354,0,0,1,0


In [219]:
df_reg = pd.merge(df[['trip_id','pickup_x','pickup_y','dropoff_x','dropoff_y',
                        'delivered_on_time','driver','pickup_time','pickup_day']], 
                    df_reg,
         on = 'trip_id').drop_duplicates('trip_id')

In [220]:
df_reg


,trip_id,pickup_x,pickup_y,dropoff_x,dropoff_y,delivered_on_time,driver,pickup_time,pickup_day,stoppage_location_1,stoppage_location_2,stoppage_location_3,stoppage_location_4
0,12,7.7,50.1,68.5,17.8,0,1,549,37,0.0,0.0,74.0,0.0
1,13,22.7,22.1,8.5,27.4,0,1,876,45,0.0,27.0,0.0,0.0
2,14,7.4,46.4,31.6,14.8,1,3,678,54,15.0,107.0,0.0,0.0
4,16,23.8,22.0,8.6,28.9,1,1,776,65,0.0,67.0,0.0,0.0
5,54,68.3,18.1,16.9,23.2,1,2,494,66,0.0,0.0,0.0,20.0
6,56,65.2,18.1,7.3,42.3,1,2,451,49,0.0,0.0,0.0,101.0
7,75,6.9,67.3,67.3,18.3,0,2,103,172,36.0,57.0,0.0,0.0
9,82,69.3,18.2,17.7,23.3,1,2,501,275,0.0,0.0,0.0,40.0
10,94,55.4,18.7,13.4,23.5,1,3,700,354,0.0,0.0,53.0,0.0


In [252]:
df_reg.loc[(df_reg['driver'] == 1) & (df_reg['delivered_on_time'] ==0),"Block"] = 1


In [253]:
df_reg


,trip_id,pickup_x,pickup_y,dropoff_x,dropoff_y,delivered_on_time,driver,pickup_time,pickup_day,stoppage_location_1,stoppage_location_2,stoppage_location_3,stoppage_location_4,Block
0,12,7.7,50.1,68.5,17.8,0,1,549,37,0.0,0.0,74.0,0.0,1.0
1,13,22.7,22.1,8.5,27.4,0,1,876,45,0.0,27.0,0.0,0.0,1.0
2,14,7.4,46.4,31.6,14.8,1,3,678,54,15.0,107.0,0.0,0.0,NaN
4,16,23.8,22.0,8.6,28.9,1,1,776,65,0.0,67.0,0.0,0.0,NaN
5,54,68.3,18.1,16.9,23.2,1,2,494,66,0.0,0.0,0.0,20.0,NaN
6,56,65.2,18.1,7.3,42.3,1,2,451,49,0.0,0.0,0.0,101.0,NaN
7,75,6.9,67.3,67.3,18.3,0,2,103,172,36.0,57.0,0.0,0.0,NaN
9,82,69.3,18.2,17.7,23.3,1,2,501,275,0.0,0.0,0.0,40.0,NaN
10,94,55.4,18.7,13.4,23.5,1,3,700,354,0.0,0.0,53.0,0.0,NaN


In [43]:
## Problem 2
#time spend in each of the stops


In [136]:
final_df_= final_df[['trip_id','distance','delivered_on_time','driver','pickup_time','pickup_day']].merge(df_, on = 'trip_id')


In [167]:
X = final_df_[['trip_id','distance','delivered_on_time','driver','pickup_time','pickup_day']]
#Y = final_df_['stoppage_location_']


In [145]:
#Y = np.vstack((final_df['stoppage_location_1'], final_df['stoppage_location_2'], final_df['stoppage_location_3'], final_df['stoppage_location_4'])).T

In [168]:
multi_target_forest.fit(X, Y).predict(X)


array([[0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]])